## 2012 rio k2 daws data - 3dB threshold check

### Function to read riometer file

##### linesplit [0] - date (dd/mm/yy)
##### linesplit [1] - time (UT)
##### linesplit [2] - absorption (dB)
##### linesplit [3] - raw signal (volts)

In [1]:
import datetime
import urllib.request

# define riometer readfile function
def rio_readfile(url):

    # Define lists
    #date = [] #do i even need the date cause it's the same year?
    time = []
    absorption = []
    raw_sig = []

    # Define filename
    #filename = "RD 2012-03-03.txt"

    # open file to read
    response = urllib.request.urlopen(url)
    html_response = response.read()
    encoding = response.headers.get_content_charset("utf-8")
    fp = html_response.decode(encoding)

    # define new list sanitized_data
    # entry = [] list defined later, append datetime, absp, raw sig to it
    # later append entry to sanitized_data so it will be lists within a list
    sanitized_data = []
    
    for line in fp.splitlines():
        #print(line)
        #print(str(line))
    
        # skip comments
        if line[0] == "#":
            continue
        else:
            
            # strip line
            line_strip = line.strip()
            # split lines into lists
            line_split = line.split()
            #print(line_split[1])

            # Define datetime format for date and time
            
            format = "%d%m%Y%H:%M:%S"
            
            # split column 0 to month, date, year and make one row
            month = str(line_split[0].split("/")[1])
            day = str(line_split[0].split("/")[0])
            year = "20" + str(line_split[0].split("/")[2])
                      
            # change this to dd mm yy!!!!!!!!!!!
            full_date = day + month + year + str(line_split[1])
            
            # Try-except to see if can convert to datetime
            try:
                res = bool(datetime.datetime.strptime(full_date, format))
                this_time = datetime.datetime.strptime(full_date, format)

                # Get rid of negative absorption values
                if float(line_split[2]) < 0:
                    continue
                    
                # Only append line to list if time checks true

                # append time to array
                time.append(this_time)
                # append absorption to array
                absorption.append(float(line_split[2]))
                # append raw signals to array
                raw_sig.append(line_split[3])

                # define new list 
                entry = []
                entry.append(this_time)
                entry.append(line_split[2])
                entry.append(line_split[3])
                
                sanitized_data.append(entry)
                    
            except ValueError:
                res = False

    #return sanitized_data
    return sanitized_data, time, absorption, raw_sig
    #return absorption
    #return raw_sig

### Function to check days with absorption > 3dB

In [2]:

# Define function high_abs to check days that have absp >3dB
def high_abs(data):
    
    counter = 0
    index = 0
    
    # threshold measurement: how many counts of 3dB absorption needed in a day to pass the check
    thresh_meas = 30
    
    # threshold absorption: minimum dB value to meet
    thresh_abs = 3
    
    # spike absorption: dB value threshold of spike
    spike_abs = 10
    
    # for every line in data, if there's a day with thresh_mesh at >= 20 of dB >= 3, then return 1
    while index < len(data):
        for measurement in data:
            # try except error cause some absorption values are ****
            try:
                if float(measurement[1]) <= spike_abs and float(measurement[1]) >= thresh_abs:
                    counter += 1
            except:
                index += 1
                continue

        if counter > thresh_meas:
            return counter, 1
            
        index += 1
    
    return 0

#### Testing high_abs function for 2012/03/03

In [3]:
sanitized_data, time, absorption, raw_sig = rio_readfile("https://data.phys.ucalgary.ca/sort_by_project/GO-Canada/GO-Rio/txt/2012/03/02/norstar_k2_rio-daws_20120302_v01.txt")

counter, check = high_abs(sanitized_data)
print("check (0 or 1): ", counter)
count=0
for line in absorption:
    if line >= 3 and line <= 10:
        count+= 1
        
print("number of absp vals >=3dB <= 10dB: ", count)

check (0 or 1):  38
number of absp vals >=3dB <= 10dB:  38


### Function to filter out spikes

In [4]:
import matplotlib.pyplot as plt
def filter_spikes(times, absorptions, threshold, year_str, month_str, day_str, datetime_format, neighbor_count=10):
    """
    Filter out absorption spikes where the value is greater than a threshold relative to the average of its neighbors.
    """
    filtered_times = []
    filtered_absorptions = []

    for i in range(len(absorptions)):
        # Determine the range for neighbors
        start_index = max(i - neighbor_count, 0)
        end_index = min(i + neighbor_count + 1, len(absorptions))

        # Calculate the average of neighboring values
        neighbor_avg = sum(absorptions[start_index:end_index]) / (end_index - start_index)

        # Check if the current value is within the threshold relative to the neighbor average
        if abs(absorptions[i] - neighbor_avg) <= threshold:
            filtered_times.append(times[i])
            filtered_absorptions.append(absorptions[i])
    
    def plot(filtered_times, filtered_absorptions):
        title_header = "{year} {month_name} {day} Riometer absorption (dB) vs time (UTC {time}) with threshold {threshold}".format(year=year_str, month_name=month_str, day=day_str, time=datetime_format, threshold=threshold)
        plt.figure()
        plt.plot(filtered_times, filtered_absorptions)
        plt.title(title_header)
        plt.xlabel("Time (UTC) {time}".format(time=datetime_format), fontsize=10)
        plt.xticks(rotation=90)
        plt.ylabel("Absorption (dB)", fontsize=10)
        plt.show()
        #plt.savefig("output.pdf")
       
    return plot(filtered_times, filtered_absorptions)

### 2012 3dB check

#### highdb is the list that all datetimes that passed the 3dB check is stored in along with their sanitized data list + counter

#####    highdb[0] = datetime of first line in url (so first ever recorded time in a day's txt file)
#####    highdb[1] = sanitized data
#####    highdb[2] = counter

### Issue Solved <3
#### Issue to solve: highdb resets with each day I think. Check testing
##### checked testing: server val exceeded or whatever
##### possible solution: instead of putting all highdb dates to a list, put counters (value) of each day along with the datetime (key) to dictionary.
##### OR, simply remove sanitized data out of highdb

In [ ]:
from calendar import monthrange

# define base URL and base year
base = "https://data.phys.ucalgary.ca/sort_by_project/GO-Canada/GO-Rio/txt/"
year = "2012"

# new list to add days with dB >3
highdb = {}

# check every month of the year
for month in range(1,13):
    
    # change month from example "4" to "04" for April (cause that's how it is in rio data)
    month_str = str(month).zfill(2)
    
    # go through every day of the month
    # with monthrange - auto day checker (like Jan 31 days, Feb 28 days etc.)
    for day in range(1, monthrange(int(year), month)[1] +1):
        
        # change day from example "9" to "09"
        day_str = str(day).zfill(2)
        
        # define new_base to replace year, month, and date with each url
        new_base = "https://data.phys.ucalgary.ca/sort_by_project/GO-Canada/GO-Rio/txt/{year}/{month}/{day}/norstar_k2_rio-daws_{year}{month}{day}_v01.txt"
        
        try:
            # read the url, return sanitized_data list
            sanitized_data, time, absorption, raw_sig = rio_readfile(new_base.format(year = year, month = month_str, day = day_str))
        
            counter, check = high_abs(sanitized_data)
            
            if check == 1:
                highdb[sanitized_data[day][0]] = counter
                
        except Exception as e:
            continue

### Get 10 days with the highest counter values

In [ ]:
top_test_max = dict(sorted(highdb.items(), key=lambda x: x[1], reverse=True)[:10])

# list with top 10 days
dates_list = []

print("hi")
for key, value in top_test_max.items():
    dates = (key, value)
    dates_list.append(dates)


for i in range(len(dates_list)):
    ymd = dates_list[i][0].date()
    
    year_str = str(ymd.year)
    month_str = str(ymd.month).zfill(2)
    day_str = str(ymd.day).zfill(2)
    
    base_url = "https://data.phys.ucalgary.ca/sort_by_project/GO-Canada/GO-Rio/txt/{year}/{month}/{day}/norstar_k2_rio-daws_{year}{month}{day}_v01.txt"
    
    print(base_url.format(year = year_str, month = month_str, day = day_str))
    sanitized_data, time, absorption, raw_sig = rio_readfile(base_url.format(year = year_str, month = month_str, day = day_str))
    
    threshold = 0.003
    
    filter_spikes(time, absorption, threshold, year_str, month_str, day_str, "dd:mm:H", neighbor_count=10)

In [ ]:
# 2017 jan 2

sanitized_data, time, absorption, raw_sig = rio_readfile("https://data.phys.ucalgary.ca/sort_by_project/GO-Canada/GO-Rio/txt/2012/03/08/norstar_k2_rio-daws_20120308_v01.txt")

threshold = 0.003
filter_spikes(time, absorption, threshold, "2012", "Mar", "08", "dd:mm:H", neighbor_count=10)
print("Length of time list:", len(time))
print("Length of absp list:", len(absorption))

In [ ]:
# 2017 jan 2

sanitized_data, time, absorption, raw_sig = rio_readfile("https://data.phys.ucalgary.ca/sort_by_project/GO-Canada/GO-Rio/txt/2012/03/07/norstar_k2_rio-daws_20120307_v01.txt")

threshold = 0.001
filter_spikes(time, absorption, threshold, "2012", "Mar", "07", "dd:mm:H", neighbor_count=10)
print("Length of time list:", len(time))
print("Length of absp list:", len(absorption))

In [ ]:
# 2017 jan 2

sanitized_data, time, absorption, raw_sig = rio_readfile("https://data.phys.ucalgary.ca/sort_by_project/GO-Canada/GO-Rio/txt/2012/03/09/norstar_k2_rio-daws_20120309_v01.txt")

threshold = 0.001
filter_spikes(time, absorption, threshold, "2012", "Mar", "09", "dd:mm:H", neighbor_count=10)
print("Length of time list:", len(time))
print("Length of absp list:", len(absorption))

In [ ]:
# 2017 jan 2

sanitized_data, time, absorption, raw_sig = rio_readfile("https://data.phys.ucalgary.ca/sort_by_project/GO-Canada/GO-Rio/txt/2012/03/01/norstar_k2_rio-daws_20120301_v01.txt")

threshold = 0.005
filter_spikes(time, absorption, threshold, "2012", "Mar", "01", "dd:mm:H", neighbor_count=10)
print("Length of time list:", len(time))
print("Length of absp list:", len(absorption))

In [ ]:
# 2017 jan 2

sanitized_data, time, absorption, raw_sig = rio_readfile("https://data.phys.ucalgary.ca/sort_by_project/GO-Canada/GO-Rio/txt/2012/11/01/norstar_k2_rio-daws_20121101_v01.txt")

threshold = 0.001
filter_spikes(time, absorption, threshold, "2012", "Nov", "01", "dd:mm:H", neighbor_count=10)
print("Length of time list:", len(time))
print("Length of absp list:", len(absorption))

In [ ]:
# 2017 jan 2

sanitized_data, time, absorption, raw_sig = rio_readfile("https://data.phys.ucalgary.ca/sort_by_project/GO-Canada/GO-Rio/txt/2012/03/10/norstar_k2_rio-daws_20120310_v01.txt")

threshold = 0.001
filter_spikes(time, absorption, threshold, "2012", "Mar", "10", "dd:mm:H", neighbor_count=10)
print("Length of time list:", len(time))
print("Length of absp list:", len(absorption))

In [ ]:
# 2017 jan 2

sanitized_data, time, absorption, raw_sig = rio_readfile("https://data.phys.ucalgary.ca/sort_by_project/GO-Canada/GO-Rio/txt/2012/02/07/norstar_k2_rio-daws_20120207_v01.txt")

threshold = 0.005
filter_spikes(time, absorption, threshold, "2012", "Feb", "07", "dd:mm:H", neighbor_count=10)
print("Length of time list:", len(time))
print("Length of absp list:", len(absorption))

In [ ]:
# 2017 jan 2

sanitized_data, time, absorption, raw_sig = rio_readfile("https://data.phys.ucalgary.ca/sort_by_project/GO-Canada/GO-Rio/txt/2012/07/06/norstar_k2_rio-daws_20120706_v01.txt")

threshold = 0.001
filter_spikes(time, absorption, threshold, "2012", "Jul", "06", "dd:mm:H", neighbor_count=10)
print("Length of time list:", len(time))
print("Length of absp list:", len(absorption))

In [ ]:
# 2017 jan 2

sanitized_data, time, absorption, raw_sig = rio_readfile("https://data.phys.ucalgary.ca/sort_by_project/GO-Canada/GO-Rio/txt/2012/04/03/norstar_k2_rio-daws_20120403_v01.txt")

threshold = 0.005
filter_spikes(time, absorption, threshold, "2012", "Apr", "03", "dd:mm:H", neighbor_count=10)
print("Length of time list:", len(time))
print("Length of absp list:", len(absorption))

In [ ]:
# 2017 jan 2

sanitized_data, time, absorption, raw_sig = rio_readfile("https://data.phys.ucalgary.ca/sort_by_project/GO-Canada/GO-Rio/txt/2012/11/07/norstar_k2_rio-daws_20121107_v01.txt")

threshold = 0.001
filter_spikes(time, absorption, threshold, "2012", "Nov", "07", "dd:mm:H", neighbor_count=10)
print("Length of time list:", len(time))
print("Length of absp list:", len(absorption))

### Function to get 1st index (so 2nd element) of lists within a list

In [ ]:
def get_first(seq):
    if isinstance(seq, (tuple, list)):
        return get_first(seq[1])
    return seq

def get_zero_list(seq):
    return [get_first(i) for i in seq]

### 2017, Jan 2 - time vs absorption

In [ ]:
# 2017 jan 2

sanitized_data, time, absorption, raw_sig = rio_readfile("https://data.phys.ucalgary.ca/sort_by_project/GO-Canada/GO-Rio/txt/2012/03/08/norstar_k2_rio-daws_20120308_v01.txt")

threshold = 0.005
filter_spikes(time, absorption, threshold, "2012", "Mar", "08", "mm:dd:H", neighbor_count=10)
print("Length of time list:", len(time))
print("Length of absp list:", len(absorption))

# Testing

In [ ]:
'''
highdb = {}
for day in range(1, 12):
    day_str = str(day).zfill(2)
    
    new_base = "https://data.phys.ucalgary.ca/sort_by_project/GO-Canada/GO-Rio/txt/{year}/{month}/{day}/norstar_k2_rio-daws_{year}{month}{day}_v01.txt"
    
    try:
        
        sanitized_data, time, absorption, raw_sig = rio_readfile(new_base.format(year="2012", month="03", day=day_str))
        
        counter, check = high_abs(sanitized_data)
        
        if check == 1:
            highdb[sanitized_data[day][0]] = counter
                
    except Exception as e:
        continue
        
print(highdb)
'''

In [ ]:
#print(len(highdb))

In [ ]:
'''

top_10_max = dict(sorted(highdb.items(), key=lambda x: x[1], reverse=True)[:10])

print("top 10 vals:")
for key, value in top_10_max.items():
    print(key, value)
    
'''

In [ ]:
'''
data = {'a': 10, 'b': 30, 'c': 0, 'd': 40, 'e': 33, 'f': 99}

# Get the top 3 maximum values using the max() function
top_3_max = dict(sorted(data.items(), key=lambda x: x[1], reverse=True)[:3])

print("Top 3 maximum values:")
for key, value in top_3_max.items():
    print(key + ":", value)
'''

In [ ]:
'''
data = {'a': 10, 'b': 30, 'c': 0, 'd': 40, 'e': 33, 'f': 99, 'g': 99}

# Get the top 3 maximum values
top_3_max_values = sorted(data.values(), reverse=True)[:3]

print("Top 3 maximum values:", top_3_max_values)
'''

In [ ]:
# Chatgpt function
'''
# Define function to check for high absorption values
def high_abs(data):
    """
    Check if there are high absorption values (> 3dB) in the data.
    
    Args:
    - data: list of absorption values
    
    Returns:
    - 1 if high absorption values are found, 0 otherwise
    """
    thresh_meas = 30  # threshold measurement: how many counts of 3dB absorption needed in a day to pass the check
    thresh_abs = 3  # threshold absorption: minimum dB value to meet
    window_size = 5  # size of the window to consider for spike detection
    spike_threshold = 20  # threshold for detecting a spike
    
    for i, measurement in enumerate(data):
        # try except error cause some absorption values are "****"
        try:
            measurement_str = str(measurement)
            abs_value = float(measurement_str)
            if abs_value >= thresh_abs:
                # Check if the current value is a spike
                if has_spike(data, i, window_size, spike_threshold):
                    continue  # Skip if it's a spike
                else:
                    counter += 1
        except ValueError:
            continue
        
        if counter > thresh_meas:
            return 1
    
    return 0
'''

### Function to filter out spikes in data set when looking for >3dB values

In [ ]:
'''
# Define function to check for spikes in absorption values
def has_spike(data, index, window_size=5, spike_threshold=10):
    """
    Check if there is a spike in absorption values around the given index.
    
    Args:
    - data: list of absorption values
    - index: index of the absorption value to check for spike
    - window_size: size of the window around the index to consider for spike detection
    - spike_threshold: threshold for detecting a spike
    
    Returns:
    - True if a spike is detected, False otherwise
    """
    start_index = max(0, index - window_size)
    end_index = min(len(data), index + window_size + 1)
    neighbors = data[start_index:end_index]
    spike_count = sum(1 for value in neighbors if value >= spike_threshold)
    return spike_count > 0
'''